# Classification Experiment on Kumar's EEG Imagined speech dataset

Run this experiment to obtain a similar result as the one publisher in the paper

```latex
@article{gallo2024eeg,
  title={Thinking is Like a Sequence of Words},
  author={Gallo, Ignzio and Coarsh, Silvia},
  journal={IJCNN},
  volume={??},
  pages={??--??},
  year={2024},
  publisher={IEEE}
}
```

## Import libraries

In [1]:
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import yaml
import mne
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm

In [2]:
def create_save_dir(args):
    if "subject_num" in args:
        if type(args['subject_num']) is not list:
            args['subject_num'] = [args['subject_num']]

        subjs_str = ','.join(str(x) for x in args['subject_num'])
        args['save_dir'] = os.path.join(args['save_dir'], subjs_str, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    else:
        args['save_dir'] = os.path.join(args['save_dir'], datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    if not os.path.isdir(args['save_dir']):
        os.makedirs(args['save_dir'])

In [3]:
class Metrics:
    def __init__(self, column_names):
        column_names.insert(0, "time_stamp")
        self.df = pd.DataFrame(columns=column_names)

    def add_row(self, row_list):
        row_list.insert(0, str(dt.datetime.now()))
        # print(row_list)
        self.df.loc[len(self.df)] = row_list

    def save_to_csv(self, filepath):
        self.df.to_csv(filepath, index=False)

## Load raw dataset

In [4]:
def class2index(args):
    basename = os.path.basename(os.path.normpath(args['data_dir']))
    if basename ==  'Digit':
        cls_names = ['0','1','2','3','4','5','6','7','8','9']
    elif basename == 'Char':
        cls_names = ['a','c','f','h','j','m','p','s','t','y']
    elif basename == 'Image':
        cls_names = ['apple','car','dog','gold','mobile','rose','scooter','tiger','wallet','watch']
    else:
        raise Exception("Sorry, unrecognized dir name:" + basename) 
    class_to_index_dic = dict(zip(cls_names, range(len(cls_names))))
    print(class_to_index_dic)
    return class_to_index_dic

def file_name_to_cls_index(file_name, class_to_index_dic):
    name, _ = file_name.split('.')
    _,cls = name.split('_')
    return class_to_index_dic[cls.lower()]    

In [5]:
def read_kumar_raw(args, unsupervised=False):
    print("Dataset dir:", args['data_dir'])
    files = os.listdir(args['data_dir'])
    num_raw_files=len(files)

    # mapping class name to indices
    charcls2index = class2index(args)

    # some statistics on all the raw files
    ts_len = []
    ts_max = []
    for fi in files:
        data = mne.io.read_raw_edf(os.path.join(args['data_dir'],fi), verbose='CRITICAL')
        # read channels 'AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'RAW_CQ'
        raw_data = data[2:16][0]
        n_channels, len_time_series = raw_data.shape
        ts_len.append(len_time_series)
        ts_max.append(np.max(raw_data))
    print(f"Lenght Time series: MIN={np.min(ts_len)}, MAX={np.max(ts_len)}")
    print(f"Max value in Time series: {np.max(ts_max)}")
    print(f"Num raw files: {num_raw_files}")
    print(f"Num channels: {n_channels}")

    # Read data and scale them
    MAX_LEN_TS = 1280
    SCALE = 1000
    assert MAX_LEN_TS <= np.min(ts_len), f"{MAX_LEN_TS} is not <= {np.min(ts_len)}"

    X = np.zeros((num_raw_files,n_channels,MAX_LEN_TS))
    Y = np.zeros((num_raw_files,))
    for idx, fi in enumerate(files):
        data = mne.io.read_raw_edf(os.path.join(args['data_dir'], fi), verbose='CRITICAL')
        raw_data_scaled = data[2:16][0]*SCALE
        X[idx,:,:] = raw_data_scaled[:,0:MAX_LEN_TS]
        Y[idx] = file_name_to_cls_index(fi, charcls2index)

    # sliding window on the source timeseries
    npt = args['vocab_size']
    X_new = np.zeros((36110, npt, n_channels))
    Y_new = np.zeros((36110,))

    stride = 8
    ctr = 0
    for i in range(0,num_raw_files):
        y = Y[i]
        a= X[i,:,:]
        a = a.transpose()
        val = 0
        while val<=(len(a)-npt):
            x = a[val:val+npt,:]
            X_new[ctr,:,:] = x
            Y_new[ctr] = y
            val = val+stride
            ctr = ctr+1          

    mean, std = X_new.mean(), X_new.std()
    print("Input data: mean =", mean, ", std =", std) # used for input data Normalization
    print("Kumar Dataset shape:", X_new.shape, ", min:", np.min(X_new), ", max:", np.max(X_new))

    # Random split. It is better to split the 230 files before the sliding window...
    X_train, X_test, Y_train, Y_test = train_test_split(np.float32(X_new), Y_new.astype(int), test_size=0.2, random_state=1)   

    # Convert NumPy arrays to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    Y_train = torch.tensor(Y_train, dtype=torch.long)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    Y_test = torch.tensor(Y_test, dtype=torch.long)
    print("Training shape:", X_train.shape)
    print("Test shape:", X_test.shape)

    # Convert data to DataLoader
    train_dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True)
    test_dataset = TensorDataset(X_test, Y_test)
    test_loader = DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False)
    return train_loader, test_loader

## The model

Deep neural network based on a basic Transformer.

In [6]:
class NetTraST(nn.Module):
    def __init__(self, args): 
        super(NetTraST, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(args['vocab_size'])
        p = args['kernel_size'] // 2
        self.conv1 = nn.Conv1d(in_channels=args['vocab_size'], out_channels=args['kernel_num'], kernel_size=args['kernel_size'], stride=1, padding=p)
        
        #self.conv2 = nn.Conv1d(in_channels=args['embed_dim'], out_channels=args['kernel_num'], kernel_size=args['kernel_size'], stride=1, padding=p)
        self.upsamp = nn.Upsample((args['embed_dim']))
        
        self.rrelu = nn.RReLU(0.1, 0.3)
        nl=3 
        self.spatial_tra = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['embed_dim'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=nl,
        )
        #self.temporal_tra = nn.TransformerEncoder(
        #    nn.TransformerEncoderLayer(
        #        d_model=args['vocab_size'],
        #        nhead=args['nhead'],
        #        dim_feedforward=args['dim_feedforward'],
        #    ),
        #    num_layers=nl,
        #)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['kernel_num'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=args['num_layers'],
        )
        self.batch_norm3 = nn.BatchNorm1d(args['kernel_num'])
        self.fl = nn.Flatten()
        self.fc1 = nn.Linear(args['kernel_num']*args['embed_dim'], args['kernel_num'])
        self.dropout = nn.Dropout(args['dropout'])
        self.fc2 = nn.Linear(args['kernel_num'], args['class_num'])

    def forward(self, x): 
        x = self.batch_norm1(x)
        
        x1 = self.conv1(x) 
        x1 = self.spatial_tra(x1)
        
        #x2 = x.permute(0, 2, 1)
        #x2 = self.conv2(x2) 
        #x2 = self.temporal_tra(x2)
        #x2 = self.upsamp(x2)

        x = x1 #x1+x2 
        #x = torch.cat((x1, x2), 1)
        
        # Reshape the input for the Transformer layer
        x = x.permute(2, 0, 1)  # Change the shape to (sequence_length, batch_size, input_size)
        x = self.transformer(x)
        # Reshape the output back to the original shape
        x = x.permute(1, 2, 0)  # Change the shape to (batch_size, input_size, sequence_length)
        x = self.batch_norm3(x)
        x = self.fl(x)
        x = self.rrelu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

## Training and Evaluation functions

In [7]:
def evaluation_raw(args, model, test_loader, criterion):
    # Evaluation
    model.eval()
    with torch.no_grad():
        tot_loss = 0
        test_corrects = torch.tensor(0, device=args['device'])
        for inputs, labels in test_loader:
            inputs = inputs.to(args['device'])
            labels = labels.to(args['device'])
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            corrects = (torch.max(outputs, 1)[1].view(labels.size()).data == labels.data).sum()
            test_corrects += corrects
            tot_loss += loss

        ts_acc = 100.0 * test_corrects/len(test_loader.dataset)
        #print(f'Test Accuracy: {ts_acc:.4f}, Test loss: {tot_loss:.6f}')  
    return ts_acc.cpu().item(), tot_loss


def train_raw(args, model, train_loader, optimizer, criterion, test_loader, subj, scheduler=None): 
    metrics = Metrics(["epoch", "lr", "train_loss", "train_acc", "test_loss", "test_acc", "best_test_acc"])
    # Training loop
    best_acc = 0
    patience_counter = 0
    steps = 0
    loop_obj = tqdm(range(args['epochs']))
    loop_obj.set_postfix_str(f"Best val. acc.: {best_acc:.4f}")  # Adds text after progressbar
    for epoch in loop_obj:
        loop_obj.set_description(f"Subj.: {subj}, Training epoch: {epoch+1}")  # Adds text before progessbar
        train_corrects = torch.tensor(0, device=args['device'])
        tot_loss = 0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs = inputs.to(args['device'])

            labels = labels.to(args['device'])
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            corrects = (torch.max(outputs, 1)[1].view(labels.size()).data == labels.data).sum()
            train_corrects += corrects
            tot_loss += loss
            loss.backward()
            optimizer.step()

        if scheduler: scheduler.step()

        tr_acc = 100.0 * train_corrects/len(train_loader.dataset)
        # Validation
        dev_acc, test_loss = evaluation_raw(args, model, test_loader, criterion)

        if dev_acc > best_acc:
            best_acc = dev_acc
            patience_counter = 0
            loop_obj.set_postfix_str(f"Best val. acc.: {best_acc:.4f}")
        else:
            patience_counter += 1

        EP=args['epochs']
        #print(f'Epoch [{epoch+1}/{EP}] Tr. Loss: {tot_loss.item():.4f} Val. Accuracy: {dev_acc:.4f} Best Val. Accuracy: {best_acc:.4f}')
        lr=optimizer.param_groups[0]["lr"]
        metrics.add_row([epoch+1, lr, tot_loss.cpu().item(), tr_acc.cpu().item(), test_loss.cpu().item(), dev_acc, best_acc])
        metrics.save_to_csv(os.path.join(args['save_dir'], "metrics_classifciation.csv"))

        if patience_counter >= args['early_stopping_patience']:
            print("Early stopping...")
            break

## Run an experiment

- change the parameter '*data_dir*' in the *args* dictionary to change the subject to one of the following 
  - /kaggle/input/kumars-eeg-imagined-speech/Imagined_speech_EEG_edf/Digit/ 
  - /kaggle/input/kumars-eeg-imagined-speech/Imagined_speech_EEG_edf/Char/
  - /kaggle/input/kumars-eeg-imagined-speech/Imagined_speech_EEG_edf/Image/
- remenber to change also the '*save_dir*' parameter

In [8]:
def get_default_args():
    args = {
        'class_num': 10,
        'dropout': 0.1 ,
        'nhead': 2 ,
        'dim_feedforward': 256 ,
        'num_layers': 5 ,
        'embed_dim': 14,
        'vocab_size': 32,
        'kernel_num': 128,
        'kernel_size': 3, 
        'batch_size': 256, # use low batch size
        'epochs': 1000 ,
        'early_stopping_patience': 300,
        'lr': 0.9 ,
        'log_interval': 1,
        'device': 'cuda:1' if torch.cuda.is_available() else 'cpu',
        'data_dir': '/home/jovyan/nfs/igallo/datasets/EEG/thoughtvizdataset/raw/', # Digit/, Char/, Image/
        'save_dir': 'experiments/transformer/', # char_raw, digit_raw, image_raw
        'save_best': True,
        'verbose': True,
        'test_interval': 100,
        'save_interval': 500,
        'sampling_rate': 128,
    }
    #args['device'] = 'cuda' if torch.cuda.is_available() else 'cpu' 
    return args

### Ablation 2
- NO Conv2
- NO Temporal transformer T2

In [9]:
def single_run():
    subjects = ["Char", "Digit", "Image"]
    for subj in subjects:
        print("Subject:", subj)
        args = get_default_args()
        args['data_dir'] = os.path.join(args['data_dir'], subj)
        args['save_dir'] = os.path.join(args['save_dir'], f"{subj.lower()}_raw")
        create_save_dir(args)

        model = NetTraST(args)
        model = model.to(args['device'])

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters()) #, lr=args['lr'])

        with open(os.path.join(args['save_dir'], "config.yaml"), "w") as f:
            yaml.dump(
                args, stream=f, default_flow_style=False, sort_keys=False
            )

        train_loader, test_loader = read_kumar_raw(args) 
        print("Training size:", len(train_loader.dataset))
        print("Test size:", len(test_loader.dataset))

        train_raw(args, model, train_loader, optimizer, criterion, test_loader, subj) 
        #torch.save(model, os.path.join(args['save_dir'], f"{model.__class__.__name__}_model_last.pt"))

In [10]:
single_run()

Subject: Char
Dataset dir: /home/jovyan/nfs/igallo/datasets/EEG/thoughtvizdataset/raw/Char
{'a': 0, 'c': 1, 'f': 2, 'h': 3, 'j': 4, 'm': 5, 'p': 6, 's': 7, 't': 8, 'y': 9}
Lenght Time series: MIN=1536, MAX=1920
Max value in Time series: 0.007820512820512819
Num raw files: 230
Num channels: 14
Input data: mean = 4.228270597865398 , std = 0.119855313949292
Kumar Dataset shape: (36110, 32, 14) , min: 0.24717948717948718 , max: 7.820512820512819
Training shape: torch.Size([28888, 32, 14])
Test shape: torch.Size([7222, 32, 14])
Training size: 28888
Test size: 7222


Subj.: Char, Training epoch: 1000: 100%|██████████| 1000/1000 [1:51:33<00:00,  6.69s/it, Best val. acc.: 88.1750]


Subject: Digit
Dataset dir: /home/jovyan/nfs/igallo/datasets/EEG/thoughtvizdataset/raw/Digit
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
Lenght Time series: MIN=1408, MAX=2304
Max value in Time series: 0.00795179487179487
Num raw files: 230
Num channels: 14
Input data: mean = 4.227561717040576 , std = 0.1405851507130073
Kumar Dataset shape: (36110, 32, 14) , min: 0.41128205128205125 , max: 7.9517948717948705
Training shape: torch.Size([28888, 32, 14])
Test shape: torch.Size([7222, 32, 14])
Training size: 28888
Test size: 7222


Subj.: Digit, Training epoch: 1000: 100%|██████████| 1000/1000 [2:01:41<00:00,  7.30s/it, Best val. acc.: 87.8289]


Subject: Image
Dataset dir: /home/jovyan/nfs/igallo/datasets/EEG/thoughtvizdataset/raw/Image
{'apple': 0, 'car': 1, 'dog': 2, 'gold': 3, 'mobile': 4, 'rose': 5, 'scooter': 6, 'tiger': 7, 'wallet': 8, 'watch': 9}
Lenght Time series: MIN=1536, MAX=2816
Max value in Time series: 0.00806
Num raw files: 230
Num channels: 14
Input data: mean = 4.22922741685585 , std = 0.13057505350656873
Kumar Dataset shape: (36110, 32, 14) , min: 0.5153846153846153 , max: 8.059999999999999
Training shape: torch.Size([28888, 32, 14])
Test shape: torch.Size([7222, 32, 14])
Training size: 28888
Test size: 7222


Subj.: Image, Training epoch: 1000: 100%|██████████| 1000/1000 [1:17:40<00:00,  4.66s/it, Best val. acc.: 85.7380]
